In [1]:
import os
import json
import pprint
import re
from Prompts import *
import pickle as pkl
import yaml
from utils.config import *
from utils.models import *
import copy
import subprocess

### Initialize Model

In [2]:
model = Model(config = dict(
    MODEL_NAME = MODEL
))

Using gpt model


### Query for Scenario

In [3]:
with open(os.path.join('locations',LOCATION,'scene_graph.json'),'r') as f:
    scene_graph = json.load(f)
scQ = ScenarioQuery()
scQ_full_prompt = scQ.get_full_prompt(context=CONTEXT,
    task=TASK,
    graph=scene_graph
)
# print(scQ_txt_payload[1]['content'][0]['text'])
# print(f"""Num tokens from main message: {utils.num_tokens_from_messages(scQ_txt_payload[1],MODEL)}""")

In [4]:
scene_graph

{'nodes': [{'type': 'narrow passage', 'id': 'Aisle1 (1)'},
  {'type': 'intersection', 'id': 'Aisle2 (2)'},
  {'type': 'open space', 'id': 'Aisle3 (3)'},
  {'type': 'wide passage', 'id': 'Aisle (4)'},
  {'type': 'open space', 'id': 'Hall (5)'},
  {'type': 'narrow passage', 'id': 'Aisle6 (6)'},
  {'type': 'narrow passage', 'id': 'Aisle7 (7)'}],
 'links': ['Aisle1 (1)<->Hall (5)',
  'Aisle2 (2)<->Aisle3 (3)',
  'Aisle2 (2)<->Hall (5)',
  'Aisle3 (3)<->Hall (5)',
  'Aisle (4)<->Hall (5)',
  'Hall (5)<->Aisle6 (6)',
  'Hall (5)<->Aisle7 (7)']}

### Get Scenario Proposal from LLM

In [4]:
model.get_payload(content = scQ_full_prompt)

[{'role': 'system',
  'content': 'You are a helpful assistant designed to output JSON'},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': ' Social navigation involves a robot navigating to a goal in the presence of humans and adhering to the following principles:\n    Safety (P1): Ensure no harm comes to people or property by avoiding collisions and risky behaviors.\n    Comfort (P2): Maintain personal space and move in ways that do not startle or distress individuals.\n    Legibility (P3): Make the robot’ s actions predictable and understandable to those around it.\n    Politeness (P4): Navigate with courtesy, often signaling intent and yielding way as needed.\n    Social Competency (P5): Adhere to social norms and cultural expectations in shared environments. Adhering to Cultural norms is also very important and included within this.\n    Understanding Other Agents (P6): Recognize and appropriately respond to the behaviors and intentions of others.\n    Proactivity (P7): 

In [5]:
if QUERY_SC:
    payload = model.get_payload(content = scQ_full_prompt)
    response = model.get_response(messages = payload,format = "json_object")
    scq_response_json = json.loads(response)
    if SAVE_SC_RESPONSE:
        with open('responses/reponse_sc.json','w') as f:
            json.dump(scq_response_json,f)
else:
    assert LOAD_SC_RESPONSE == True
    with open('responses/reponse_sc.json','r') as f:
        scq_response_json = json.load(f)
print(scq_response_json.keys())
pprint.pprint(scq_response_json)

dict_keys(['Scenario Description', 'Number of Humans', 'Trajectories', 'Behaviors', 'Expected Robot Behavior', 'Principle Ranking', 'Reasoning'])
{'Behaviors': {'Human 1': 'Employee moves swiftly through the intersection but '
                          'acknowledges the presence of the robot by making '
                          'brief eye contact.'},
 'Expected Robot Behavior': 'The robot waits at the intersection Aisle2 (2) '
                            "and says 'MAKE WAY' to the employee. Once the "
                            'employee has passed, the robot proceeds to Aisle7 '
                            '(7) at a steady pace.',
 'Number of Humans': 1,
 'Principle Ranking': 'Safety (P1) > Politeness (P4) > Task Efficiency (P8) > '
                      'Social Competency (P5) > Legibility (P3) > Comfort (P2) '
                      '> Understanding Other Agents (P6) > Proactivity (P7)',
 'Reasoning': "This scenario tests the robot's ability to interact with "
              'emplo

In [6]:
print(scq_response_json.keys())
pprint.pprint(scq_response_json)

dict_keys(['Scenario Description', 'Number of Humans', 'Trajectories', 'Behaviors', 'Expected Robot Behavior', 'Principle Ranking', 'Reasoning'])
{'Behaviors': {'Human 1': 'Employee moves swiftly through the intersection but '
                          'acknowledges the presence of the robot by making '
                          'brief eye contact.'},
 'Expected Robot Behavior': 'The robot waits at the intersection Aisle2 (2) '
                            "and says 'MAKE WAY' to the employee. Once the "
                            'employee has passed, the robot proceeds to Aisle7 '
                            '(7) at a steady pace.',
 'Number of Humans': 1,
 'Principle Ranking': 'Safety (P1) > Politeness (P4) > Task Efficiency (P8) > '
                      'Social Competency (P5) > Legibility (P3) > Comfort (P2) '
                      '> Understanding Other Agents (P6) > Proactivity (P7)',
 'Reasoning': "This scenario tests the robot's ability to interact with "
              'emplo

In [7]:
if USE_HANDCRAFTED_SCENARIO:
    scenario_desc = scenario_desc_hc
    num_humans = num_humans
    traj_desc = traj_desc_hc
    behav_desc = behav_desc_hc
else:
    scenario_desc = scq_response_json['Scenario Description']
    num_humans = scq_response_json['Number of Humans']
    traj_desc = {}
    for k,v in scq_response_json['Trajectories'].items():
        traj_desc[k] = v.split(' -> ')
    behav_desc = scq_response_json['Behaviors']

# Assume the output is formatted correctly
expected_robot_behav_desc = scq_response_json['Expected Robot Behavior'] 
pranking_desc = scq_response_json['Principle Ranking'] 
reasoning_desc = scq_response_json['Reasoning'] 

In [8]:
coarse_locations = []
for k,v in traj_desc.items():
    for loc in v:
        match = re.search(r"\((\d+)\)", loc)
        if match:
            l = int(match.group(1))
            if l not in coarse_locations:
                coarse_locations.append(l)
print(f'coarse_locations:{coarse_locations}')

coarse_locations:[7, 3, 6, 5, 2]


In [9]:
coarse_loc_imgs = []
[coarse_loc_imgs.append(os.path.join('locations',LOCATION,f'{img}.png')) for img in coarse_locations]
#for loc in coarse_locations:
#    coarse_loc_imgs[loc] = utils.encode_image(os.path.join('locations',location,f'{loc}.png'))
#coarse_loc_imgs['overhead'] = utils.encode_image(os.path.join('locations',location,'overhead_annotated.png'))
#coarse_loc_imgs = 
coarse_loc_imgs.append(os.path.join('locations',LOCATION,'overhead_annotated.png'))

### Fine Locations Query

In [10]:
flocationQ = FLocationQuery(
    loc_imgs = coarse_loc_imgs
)

flq_full_prompt = flocationQ.get_full_prompt(
    scene_graph = str(scene_graph),
    num_humans = num_humans,
    traj_desc = traj_desc,
    sc_desc = scenario_desc
)

In [11]:
pprint.pprint(flq_full_prompt)

{'system': 'You are a helpful assistant designed to output JSON',
 'user': [{'content': 'locations/Warehouse/7.png',
           'detail': 'high',
           'type': 'img'},
          {'content': 'locations/Warehouse/3.png',
           'detail': 'high',
           'type': 'img'},
          {'content': 'locations/Warehouse/6.png',
           'detail': 'high',
           'type': 'img'},
          {'content': 'locations/Warehouse/5.png',
           'detail': 'high',
           'type': 'img'},
          {'content': 'locations/Warehouse/2.png',
           'detail': 'high',
           'type': 'img'},
          {'content': 'locations/Warehouse/overhead_annotated.png',
           'detail': 'high',
           'type': 'img'},
          {'content': '\n'
                      'A scenario for humans and robot is being created in a '
                      'social-navigation simulator. The location where the '
                      'scenario takes place has the following scene graph, '
               

### Get Fine Locations from LLM

In [12]:
if QUERY_LOC:
    payload = model.get_payload(content = flq_full_prompt)
    response = model.get_response(messages = payload,format = 'json_object')
    flq_response_json = json.loads(response)
    if SAVE_LOC_RESPONSE:
        with open('responses/reponse_loc.json','w') as f:
            json.dump(flq_response_json,f)
else:
    assert LOAD_LOC_RESPONSE == True
    with open('responses/reponse_loc.json','r') as f:
        flq_response_json = json.load(f)
print(flq_response_json.keys())
pprint.pprint(flq_response_json)

dict_keys(['Trajectory'])
{'Trajectory': {'Human 1': 'Aisle6-6A -> Hall-5B -> Aisle3-3B',
                'Human 2': 'Aisle6-6B -> Hall-5A -> Aisle2-2A',
                'Robot': 'Aisle7-7A -> Aisle3-3A'}}


In [13]:
trajectories = {}
for k,v in flq_response_json['Trajectory'].items():
    trajectories[k] = []
    for loc in v.split(' -> '):
        trajectories[k].append(loc.split('-')[1])
print(trajectories)

{'Robot': ['7A', '3A'], 'Human 1': ['6A', '5B', '3B'], 'Human 2': ['6B', '5A', '2A']}


In [5]:
### Get world location of the trajectories
with open(os.path.join('locations',LOCATION,'annotated_world_coords.pkl'),'rb') as f:
    flwc = pkl.load(f)
# with open(os.path.join('locations',LOCATION,'fine_loc_world_coords.pkl'),'rb') as f:
#     flwc = pkl.load(f)

In [6]:
flwc

{'1A': (6.200000000000001, 8.85),
 '1B': (2.3000000000000007, 8.8),
 '1C': (3.6000000000000014, 8.85),
 '1D': (5.300000000000001, 8.85),
 '2A': (5.050000000000001, 4.699999999999999),
 '2B': (6.25, 4.699999999999999),
 '2C': (4.200000000000001, 5.1),
 '2D': (4.200000000000001, 6.2),
 '2E': (3.5500000000000007, 4.6499999999999995),
 '2F': (2.450000000000001, 4.75),
 '2G': (4.200000000000001, 3.9499999999999993),
 '2H': (4.15, 2.9499999999999993),
 '3A': (2.25, 2.4000000000000004),
 '3B': (2.450000000000001, 1.549999999999999),
 '3C': (6.15, 2.3499999999999996),
 '3D': (6.25, 1.299999999999999),
 '3E': (4.550000000000001, 1.1500000000000004),
 '3F': (4.5, 2.55),
 '4A': (3.450000000000001, 13.9),
 '4B': (1.5999999999999996, 13.85),
 '4C': (6.25, 14.0),
 '4D': (1.5, 12.8),
 '4E': (1.4000000000000004, 12.1),
 '5A': (-0.6499999999999995, 8.65),
 '5B': (1.5, 8.9),
 '5C': (-0.2999999999999998, 7.05),
 '5D': (0.5, 6.1),
 '5E': (1.5, 7.1),
 '5F': (-1.75, 7.9),
 '6A': (-0.25, 3.5),
 '6B': (-2.8, 

annotated_positions = {}
starti = 1
for k,v in flwc.items():
    startc = 65    
    for l in v:
        annotated_positions[str(starti)+chr(startc)] = (l[0],-l[1])
        startc+=1
    starti+=1
print(annotated_positions)
with open(os.path.join('locations',LOCATION,'annotated_world_coords.pkl'),'wb') as f:
    pkl.dump(annotated_positions,f)

In [15]:
trajectories_world_coords = {}
for k,v in trajectories.items():
    trajectories_world_coords[k] = []
    for loc in v:
        trajectories_world_coords[k].append(flwc[loc])
print(trajectories_world_coords)

{'Robot': [(-0.6499999999999995, 8.15), (2.25, 2.4000000000000004)], 'Human 1': [(-0.25, 3.5), (1.5, 8.9), (2.450000000000001, 1.549999999999999)], 'Human 2': [(-2.8, 3.5), (-0.6499999999999995, 8.65), (5.050000000000001, 4.699999999999999)]}


#### Add fine locations to sim yaml files

In [16]:
agents_yaml = {'hunav_loader': {'ros__parameters': {'map': LOCATION,
   'publish_people': True,
   'agents': []}}}
blank_human = {'id': None,
    'skin': 0,
    'behavior': 0,
    'group_id': -1,
    'max_vel': 1.5,
    'radius': 0.4,
    'init_pose': {'x': None, 'y': None, 'z': 1.25, 'h': 0.0},
    'goal_radius': 0.3,
    'cyclic_goals': False,
    'goals': [],
    }
agents = {}

In [17]:
for i in range(num_humans):
    agents_yaml['hunav_loader']['ros__parameters']['agents'].append(f'agent{i}')
    agents[f'agent{i}'] = copy.deepcopy(blank_human)
    agents[f'agent{i}']['id'] = i
    agents[f'agent{i}']['behavior'] = 7+i
    for j,g in enumerate(trajectories_world_coords[f'Human {i+1}']):
        if j == 0:
            agents[f'agent{i}']['init_pose'] = {
                'x':g[0],
                'y':g[1],
                'z':1.25,
                'h':0.0,
            } 
        else:
            agents[f'agent{i}']['goals'].append(f'g{j}')
            agents[f'agent{i}'][f'g{j}'] = {
                'x':g[0],
                'y':g[1],
                'h':1.25
            }
agents_yaml['hunav_loader']['ros__parameters'].update(agents)

In [18]:
with open(HUNAV_SIM_AGENTS_FILE,'w') as f:
    yaml.dump(agents_yaml,f)

with open(os.path.join(HUNAV_GAZEBO_WRAPPER_DIR,'config','robot.yaml'),'w') as f:
    yaml.dump({
    'x_pose': trajectories_world_coords['Robot'][0][0],
    'y_pose': trajectories_world_coords['Robot'][0][1]
},f)

### Query LLM for BT for humans

In [19]:
custom_node_requests = []
for i in range(num_humans):
    btq = BTQuery()
    if QUERY_BT:
        print(behav_desc[f'Human {i+1}'])
        btq_full_prompt = btq.get_full_prompt(behavior = behav_desc[f'Human {i+1}'])
        payload = model.get_payload(content = btq_full_prompt)
        response = model.get_response(messages = payload,format = 'json_object')
        btq_response_json = json.loads(response)
        if SAVE_BT_RESPONSE:
            with open(f'responses/reponse_bt_{i+1}.json','w') as f:
                json.dump(btq_response_json,f)
    else:
        assert LOAD_BT_RESPONSE == True
        with open(f'responses/reponse_bt_{i+1}.json','r') as f:
            btq_response_json = json.load(f)
        #print(btq_response_json.keys())
    pprint.pprint(btq_response_json)
    print('----')
    bt_xml = btq_response_json['Tree']
    for k,v in btq_response_json.items():
        if 'custom' in str.lower(k):
            custom_node_requests.append(v)
    with open(os.path.join(HUNAV_SIM_BT_FOLDER,f'LLMBT_{i}.xml'),'w') as f:
        f.write(bt_xml)
    print(f"Wrote BT to LLMBT_{i}.xml")

{'Custom Action 1': 'Design an Action node called SlowNav(agent_id, t) with '
                    'the following logic: The agent navigates through the '
                    'environment at a slower pace compared to regular '
                    'navigation, taking into account reduced time steps for '
                    'movement.',
 'Reasoning': "The human's behavior should check for robot visibility within a "
              '1m range. If the robot is visible, they will perform the '
              'SurprisedNav action for 3 seconds. Otherwise, a slow navigation '
              'behavior is needed; since this behavior is not present in the '
              'provided nodes, it will have to be a custom action node named '
              'SlowNav.',
 'Tree': '<root main_tree_to_execute = "StartledSlowNavTree">\n'
         '\n'
         '    <TreeNodesModel>\n'
         '        <Action ID="SurprisedNav">\n'
         '            <input_port name="agent_id" type="int">identifier of the '
 

### Query LLM for custom Nodes and Auxillary functions

In [20]:
print(custom_node_requests)

['Design an Action node called SlowNav(agent_id, t) with the following logic: The agent navigates through the environment at a slower pace compared to regular navigation, taking into account reduced time steps for movement.']


In [21]:
for i in range(len(custom_node_requests)):
    if QUERY_AUX:
        ctnq = NodeQuery()
        ctnq_full_prompt = ctnq.get_full_prompt(description = custom_node_requests[i])
        payload = model.get_payload(content = ctnq_full_prompt)
        response = model.get_response(messages = payload,format = 'json_object')
        ctnq_response_json = json.loads(response)
        if SAVE_AUX_RESPONSE:
            with open('responses/reponse_aux.json','w') as f:
                json.dump(ctnq_response_json,f)
    else:
        assert LOAD_AUX_RESPONSE == True
        with open('responses/reponse_aux.json','r') as f:
            ctnq_response_json = json.load(f)
    with open(os.path.join('templates','extended_bt_functions.cpp'),'r') as f:
        btf_cpp = f.read()

    with open(os.path.join('templates','extended_bt_functions.hpp'),'r') as f:
        btf_hpp = f.read()

    #register node in bt_node.cpp
    with open(os.path.join('templates','extended_agent_manager.cpp'),'r') as f:
        agm_cpp = f.read()

    with open(os.path.join('templates','extended_agent_manager.hpp'),'r') as f:
        agm_hpp = f.read()

    with open(os.path.join('templates','extended_bt_node.cpp'),'r') as f:
        btn_cpp = f.read()

    with open(os.path.join('templates','extended_bt_node.hpp'),'r') as f:
        btn_hpp = f.read()
    
    #Write functions to extended_bt_functions.cpp file
    btf = ctnq_response_json['NODE_DEFINITION']
    btf = btf.replace('BT::NodeStatus BTfunctions::','BT::NodeStatus BTfunctionsExt::')
    btf_name = ctnq_response_json['NODE_NAME']
    btf_type = ctnq_response_json['NODE_TYPE'].lower().capitalize()
    btfn_name = btf_name[0].lower() + btf_name[1:]
    btf_header = ctnq_response_json['NODE_HEADER']

    btf_cpp = btf_cpp.replace('//<NEW FUNCTION>','//<NEW FUNCTION> \n' + btf)
    btf_hpp = btf_hpp.replace('//<NEW PUBLIC FUNCTION>','//<NEW PUBLIC FUNCTION> \n' + btf_header)
    
    #register BT nodes in extended_bt_node.cpp
    #   3 ports are available for each BT node:
    #       simple_port: agent_id
    #       visibleports: agent_id + distance
    #       portsNav: agent_id + timestep
    
    node_register = f"""factory_.registerSimple{btf_type}("{ctnq_response_json['NODE_NAME']}",std::bind(&BTfunctionsExt::{btfn_name},&btfunc_, _1),PORT);"""
    if ctnq_response_json['PORTS_USED'] == ['agent_id','distance']:
        node_register = node_register.replace('PORT','visibleports')
    elif ctnq_response_json['PORTS_USED'] == ['agent_id','time_step']:
        node_register = node_register.replace('PORT','portsNav')
    elif ctnq_response_json['PORTS_USED'] == ['agent_id']:
        node_register = node_register.replace('PORT','simple_port')
    else:
        node_register = node_register.replace(',PORT','')

    btn_cpp = btn_cpp.replace('//<NEW NODE REGISTER>','//<NEW NODE REGISTER> \n' + node_register)   

    #add aux functions
    for j,agmf in enumerate(ctnq_response_json['AUX_FUNCTIONS']):
        agmf = agmf.replace('void AgentManager::','void AgentManagerExt::')
        agm_cpp = agm_cpp.replace('//<NEW FUNCTION>','//<NEW FUNCTION> \n' + agmf)
        agm_hpp = agm_hpp.replace('//<NEW PUBLIC FUNCTION>','//<NEW PUBLIC FUNCTION> \n' + ctnq_response_json['AUX_FUNCTION_HEADERS'][j])

with open(os.path.join(HUNAV_SIM_CPP_FOLDER,'extended_bt_functions.cpp'),'w') as f:
    f.writelines(btf_cpp)

with open(os.path.join(HUNAV_SIM_HPP_FOLDER,'extended_bt_functions.hpp'),'w') as f:
    f.writelines(btf_hpp)

with open(os.path.join(HUNAV_SIM_CPP_FOLDER,'extended_bt_node.cpp'),'w') as f:
    f.writelines(btn_cpp)

with open(os.path.join(HUNAV_SIM_HPP_FOLDER,'extended_bt_node.hpp'),'w') as f:
    f.writelines(btn_hpp)

with open(os.path.join(HUNAV_SIM_CPP_FOLDER,'extended_agent_manager.cpp'),'w') as f:
    f.writelines(agm_cpp)

with open(os.path.join(HUNAV_SIM_HPP_FOLDER,'extended_agent_manager.hpp'),'w') as f:
    f.writelines(agm_hpp)

### Build Project

In [22]:
s = subprocess.getstatusoutput(f' cd ~/catkin_ws && colcon build')
if s[0] == 0:
    print('Build Successful')
else:
    print('Build Failed')
    print(s[1]) 

Build Successful
